### Advanced Python Constructs

In [1]:
import warnings

warnings.filterwarnings('ignore')

### Iterators, generator expressions and generators

#### Iterators

In [2]:
nums = [1, 2, 3] # note that ... varies: these are different objects
iter(nums)

In [3]:
nums.__iter__()


In [4]:
nums.__reversed__()

In [5]:
it = iter(nums)

In [6]:
next(it)

1

In [7]:
next(it)

2

In [8]:
next(it)

3

In [9]:
next(it)

StopIteration: 

In [ ]:
f = open('data/etc/fstab')
f is f.__iter__()

#### Generator expressions

In [11]:
(i for i in nums)

<generator object <genexpr> at 0x000001C0611B4970>

In [12]:
[i for i in nums]


[1, 2, 3]

In [13]:
list(i for i in nums)

[1, 2, 3]

In [14]:
{i for i in range(3)}

{0, 1, 2}

In [15]:
{i:i**2 for i in range(3)}

{0: 0, 1: 1, 2: 4}

#### Generators

In [16]:
def f():
    yield 1
    yield 2


In [17]:
f()

<generator object f at 0x000001C0611B4C10>

In [18]:
gen = f()
next(gen)


1

In [19]:
next(gen)

2

In [20]:
next(gen)

StopIteration: 

In [21]:
def f():
    print("-- start --")
    yield 3
    print("-- middle --")
    yield 4
    print("-- finished --")


In [22]:
gen = f()
next(gen)


-- start --


3

In [23]:
next(gen)


-- middle --


4

In [24]:
next(gen)

-- finished --


StopIteration: 

#### Bidirectional communication

In [ ]:
raise type, value, traceback

In [26]:
import itertools

def g():
    print('--start--')
    
    for i in itertools.count():
        print('--yielding %i --' % i)
        try:
            ans = yield i
        
        except GeneratorExit:
            print('--closing--')
            raise
    
        except Exception as e:
            print('--yield raised %r --' % e)
        
        else:
            print('--yield returned %s --' % ans)


In [27]:
it = g()
next(it)

--start--
--yielding 0 --


0

In [28]:
it.send(11)

--yield returned 11 --
--yielding 1 --


1

In [29]:
it.throw(IndexError)

--yield raised IndexError() --
--yielding 2 --


2

In [30]:
it.close()

--closing--


#### 8.1.5 Chaining generators

In [ ]:
subgen = some_other_generator()

for v in subgen:
    yield v

In [ ]:
yield from some_other_generator()

### Decorators

In [ ]:
@decorator # ·
def function(): # ¶
    pass

In [ ]:
def function(): # ¶
    pass

function = decorator(function) # ·

#### Decorators implemented as classes and as functions

In [34]:
def simple_decorator(function):
    print("doing decoration")
    
    return function

In [35]:
@simple_decorator
def function():
    print("inside function")

doing decoration


In [36]:
function()

inside function


In [38]:
def decorator_with_arguments(arg):
    print("defining the decorator")

    def _decorator(function):
        # in this inner function, arg is available too
        print("doing decoration, %r " % arg)
    
        return function
    return _decorator

In [39]:
@decorator_with_arguments("abc")
def function():
    print("inside function")

defining the decorator
doing decoration, 'abc' 


In [40]:
function()

inside function


In [41]:
def replacing_decorator_with_args(arg):
    print("defining the decorator")
    
    def _decorator(function):
        # in this inner function, arg is available too
        print("doing decoration, %r " % arg)
    
        def _wrapper(*args, **kwargs):
            print("inside wrapper, %r %r " % (args, kwargs))
            
            return function(*args, **kwargs)
    
        return _wrapper
    
    return _decorator

In [42]:
@replacing_decorator_with_args("abc")
def function(*args, **kwargs):
    print("inside function, %r %r " % (args, kwargs))
    return 14

defining the decorator
doing decoration, 'abc' 


In [43]:
function(11, 12)

inside wrapper, (11, 12) {} 
inside function, (11, 12) {} 


14

In [44]:
class decorator_class(object):
    def __init__(self, arg):
        # this method is called in the decorator expression
        print("in decorator init, %s " % arg)
        self.arg = arg
    
    
    def __call__(self, function):
        # this method is called to do the job
        print("in decorator call, %s " % self.arg)
        return function


In [45]:
deco_instance = decorator_class('foo')

in decorator init, foo 


In [46]:
@deco_instance
def function(*args, **kwargs):
    print("in function, %s %s " % (args, kwargs))

in decorator call, foo 


In [47]:
function()

in function, () {} 


In [48]:
class replacing_decorator_class(object):
    def __init__(self, arg):
        
        # this method is called in the decorator expression
        print("in decorator init, %s " % arg)
        self.arg = arg
        
    def __call__(self, function):
        # this method is called to do the job
        print("in decorator call, %s " % self.arg)
        self.function = function
        return self._wrapper
    
    def _wrapper(self, *args, **kwargs):
        print("in the wrapper, %s %s " % (args, kwargs))
        return self.function(*args, **kwargs)

In [49]:
deco_instance = replacing_decorator_class('foo')

in decorator init, foo 


In [50]:
@deco_instance
def function(*args, **kwargs):
    print("in function, %s %s " % (args, kwargs))

in decorator call, foo 


In [51]:
function(11, 12)

in the wrapper, (11, 12) {} 
in function, (11, 12) {} 


In [53]:
import functools

def replacing_decorator_with_args(arg):
    print("defining the decorator")
    
    def _decorator(function):
        print("doing decoration, %r " % arg)
        
        def _wrapper(*args, **kwargs):
            print("inside wrapper, %r %r " % (args, kwargs))
            return function(*args, **kwargs)
    
        return functools.update_wrapper(_wrapper, function)
    
    return _decorator


In [54]:
@replacing_decorator_with_args("abc")
def function():
    "extensive documentation"
    print("inside function")
    return 14

defining the decorator
doing decoration, 'abc' 


In [55]:
function
print(function.__doc__)

extensive documentation


#### Examples in the standard library

In [56]:
class Array(object):
    def __init__(self, data):
        self.data = data
    
    
    @classmethod
    def fromfile(cls, file):
        data = numpy.load(file)
        
        return cls(data)

In [58]:
class A(object):

    @property
    def a(self):
        "an important attribute"
        
        return "a value"

In [59]:
A.a
A().a

'a value'

In [60]:
class Rectangle(object):
    def __init__(self, edge):
        self.edge = edge


    @property
    def area(self):
        """Computed area.
        Setting this updates the edge length to the proper value.
        """
    
        return self.edge**2


    @area.setter
    def area(self, area):
        self.edge = area ** 0.5

In [61]:
class D(object):
    @property
    def a(self):
        print("getting 1")
        return 1


    @a.setter
    def a(self, value):
        print("setting %r " % value)


    @a.deleter
    def a(self):
        print("deleting")


In [62]:
D.a

In [63]:
D.a.fget

<function __main__.D.a(self)>

In [64]:
D.a.fset

<function __main__.D.a(self, value)>

In [65]:
D.a.fdel

<function __main__.D.a(self)>

In [66]:
d = D() # ... varies, this is not the same `a` function
d.a

getting 1


1

In [67]:
d.a = 2

setting 2 


In [68]:
del d.a

deleting


In [69]:
d.a

getting 1


1

#### Deprecation of functions

In [70]:
class deprecated(object):
    """Print a deprecation warning once on first use of the function.
    @deprecated() # doctest: +SKIP
    def f():
        pass
    
    f() # doctest: +SKIP
    f is deprecated
    """

In [71]:
def __call__(self, func):
    self.func = func
    self.count = 0
    return self._wrapper


def _wrapper(self, *args, **kwargs):
    self.count += 1
    if self.count == 1:
        print(self.func.__name__, 'is deprecated')
    
    return self.func(*args, **kwargs)

In [72]:
def deprecated(func):
    """Print a deprecation warning once on first use of the function."""

In [73]:
@deprecated # doctest: +SKIP
def f():
    pass


In [76]:
'f() # doctest: +SKIP f is deprecated'
count = [0]
def wrapper(*args, **kwargs):
    count[0] += 1
    if count[0] == 1:
        print(func.__name__, 'is deprecated')
    
    return

In [ ]:
def find_answers():
    answers = []
    while True:
        ans = look_for_next_answer()
        if ans is None:
            break
        answers.append(ans)
        
    return answers

In [77]:
def vectorized(generator_func):
    def wrapper(*args, **kwargs):
    
        return list(generator_func(*args, **kwargs))
    
    return functools.update_wrapper(wrapper, generator_func)

In [78]:
@vectorized
def find_answers():
    while True:
        ans = look_for_next_answer()
        if ans is None:
            break
        
        yield ans

In [79]:
class WordProcessor(object):
    PLUGINS = []
    
    def process(self, text):
        for plugin in self.PLUGINS:
            text = plugin().cleanup(text)
        
        return text
    
    
    @classmethod
    def plugin(cls, plugin):
        cls.PLUGINS.append(plugin)
    
    
@WordProcessor.plugin
class CleanMdashesExtension(object):
    def cleanup(self, text):
        
        return text.replace('&mdash;', u'\N{em dash}')

### Context managers

In [ ]:
with manager as var:
do_something(var)

In [ ]:
var = manager.__enter__()
try:
do_something(var)
finally:
manager.__exit__()

In [ ]:
class closing(object):
def __init__(self, obj):
self.obj = obj
def __enter__(self):
return self.obj
def __exit__(self, *args):
self.obj.close()


In [ ]:
with closing(open('/tmp/file', 'w')) as f:
    f.write('the contents\n')

In [ ]:
with open('/tmp/file', 'a') as f:
    f.write('more contents\n')

#### Catching exceptions

In [ ]:
class assert_raises(object):
# based on pytest and unittest.TestCase
def __init__(self, type):
self.type = type
def __enter__(self):
pass
def __exit__(self, type, value, traceback):
if type is None:
raise AssertionError('exception expected')
if issubclass(type, self.type):
return True # swallow the expected exception
raise AssertionError('wrong exception type')
with assert_raises(KeyError):
{}['foo']

#### Using generators to define context managers

In [ ]:
@contextlib.contextmanager
def some_generator(<arguments>):
<setup>
try:
yield <value>
finally:
<cleanup>

In [ ]:
@contextlib.contextmanager
def closing(obj):
try:
yield obj
finally:
obj.close()

In [ ]:
@contextlib.contextmanager
def assert_raises(type):
try:
yield
except type:
return
except Exception as value:
raise AssertionError('wrong exception type')
else:
raise AssertionError('exception expected')

In [ ]:
%reload_ext watermark
%watermark -a "Caique Miranda" -gu "caiquemiranda" -iv

Author: Caique Miranda

Github username: caiquemiranda

sys: 3.10.5 (tags/v3.10.5:f377153, Jun  6 2022, 16:14:13) [MSC v.1929 64 bit (AMD64)]




### End.